In [2]:
import syft as sy
import torch as th
from torch import nn, optim

In [3]:
hook = sy.TorchHook(th)

bob           = sy.VirtualWorker(hook, id='bob')
alice         = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [4]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])
print('ok')

W0721 13:20:27.937804 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0721 13:20:27.940310 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0721 13:20:27.943230 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


W0721 13:20:27.947031 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0721 13:20:27.949030 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0721 13:20:27.951304 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


In [5]:
# Dataset
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

data   = th.tensor([[0.,0], [0,1], [1,0], [1,1]], requires_grad=True)
target = th.tensor([[0.], [0], [1], [1]], requires_grad=True)

bobs_data   = data[:2].send(bob)
bobs_target = target[:2].send(bob)

alices_data   = data[2:].send(alice)
alices_target = target[2:].send(alice)


In [6]:
def model_learning(obj_model, obj_data, obj_target):
    obj_opt  = optim.SGD(params=obj_model.parameters(), lr=0.1)
    
    obj_opt.zero_grad()
    obj_pred = obj_model(obj_data)

    obj_loss = ((obj_pred - obj_target)**2).sum()
    obj_loss.backward()
    
    obj_opt.step()
    
    return obj_loss.get().data

# model
model = nn.Linear(2,1)

Bob loss: tensor(9.7848e-06) Alice loss: tensor(1.6426e-06)
bobs_model.parameters(): Parameter>[PointerTensor | me:26892742244 -> bob:24956531739] Parameter>[PointerTensor | me:67481676520 -> bob:97591346447]


TypeError: print() argument after * must be an iterable, not Tensor

In [27]:

 for round_iter in range(10):
    bobs_model   = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bob_loss   = th.FloatTensor
    alice_loss = th.FloatTensor

    for i in range(10):
        bob_loss   = model_learning(bobs_model,   bobs_data,   bobs_target)
        alice_loss = model_learning(alices_model, alices_data, alices_target)
    else:
        print('Bob loss:', str(bob_loss), 'Alice loss:', str(alice_loss))

    print('bobs_model.parameters():', *bobs_model.parameters())
    for p in [*bobs_model.parameters()]:
        print('p:', p)
        p.fix_prec() #.share(secure_worker)
    
    bobs_model.move(secure_worker)
    alices_model.move(secure_worker)

    with th.no_grad():
        model.weight.set_( ((alices_model.weight.data + bobs_model.weight.data)/2).get() )
        model.bias.set_(   ((alices_model.bias.data + bobs_model.bias.data)/2).get()     )

Bob loss: tensor(9.7848e-06) Alice loss: tensor(1.6426e-06)
bobs_model.parameters(): Parameter>[PointerTensor | me:26892742244 -> bob:24956531739] Parameter>[PointerTensor | me:67481676520 -> bob:97591346447]


TypeError: print() argument after * must be an iterable, not Tensor

Bob loss: tensor(9.7848e-06) Alice loss: tensor(1.6426e-06)
bobs_model.parameters(): Parameter>[PointerTensor | me:26892742244 -> bob:24956531739] Parameter>[PointerTensor | me:67481676520 -> bob:97591346447]


TypeError: print() argument after * must be an iterable, not Tensor

In [10]:
bob_model   = nn.Linear(2,1)
alice_model = nn.Linear(2,1)

bob_model.share(bob, alice,   crypto_provider=secure_worker)
alice_model.share(bob, alice, crypto_provider=secure_worker)

print('secure_worker._objects:', secure_worker._objects)
 
for round_iter in range(10):
    bob_loss   = th.FloatTensor
    alice_loss = th.FloatTensor

    for i in range(10):
        bob_loss   = model_learning(bob_model, bobs_data,   bobs_target).float_precision()
        alice_loss = model_learning(alice_model, alices_data, alices_target).float_precision()
    else:
        print('Bob loss:', str(bob_loss), 'Alice loss:', str(alice_loss))

bob_model   = bob_model.get()
alice_model = alice_model.get()

bob_model.weight.fix_precision().share(bob, alice,   crypto_provider=secure_worker)
bob_model.bias.fix_precision().share(bob,   alice,   crypto_provider=secure_worker)

alice_model.weight.fix_precision().share(bob, alice, crypto_provider=secure_worker)
alice_model.bias.fix_precision().share(bob,   alice, crypto_provider=secure_worker)

print('weight after:', model.weight)
print('bias after:', model.bias)

secure_worker._objects: {}


AttributeError: 'AdditiveSharingTensor' object has no attribute 'attr'

In [1]:
with th.no_grad():
    pass